In [1]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD')
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Technical Assessement THD


In [2]:
import pandas as pd
import numpy as np
import datetime


In [3]:
df=pd.read_csv('store_sales_data_final.csv')


df_base=df.drop(['store_info','holiday_info'],axis=1)
df_base

,date,store_nbr,sales,promotion_count
0,2013-01-01,1,0.000000,0
1,2013-01-02,1,7417.148000,0
2,2013-01-03,1,5873.244001,0
3,2013-01-04,1,5919.879001,0
4,2013-01-05,1,6318.785010,0
...,...,...,...,...
90931,2017-08-11,54,8513.834000,166
90932,2017-08-12,54,9139.678002,138
90933,2017-08-13,54,14246.827996,173
90934,2017-08-14,54,11882.994000,126


In [4]:


df1=df[['date','store_nbr','store_info']].reset_index(drop=True)

df1['store_info']=df1['store_info'].map(eval)


df_store = pd.json_normalize(df1['store_info'])

df1=pd.concat([df1,df_store],axis=1)

processed_store=df1.drop(['store_info'],axis=1)

processed_store



,date,store_nbr,city,state,type,CLUSTER
0,2013-01-01,1,Quito,Pichincha,D,13
1,2013-01-02,1,Quito,Pichincha,D,13
2,2013-01-03,1,Quito,Pichincha,D,13
3,2013-01-04,1,Quito,Pichincha,D,13
4,2013-01-05,1,Quito,Pichincha,D,13
...,...,...,...,...,...,...
90931,2017-08-11,54,El Carmen,Manabi,C,3
90932,2017-08-12,54,El Carmen,Manabi,C,3
90933,2017-08-13,54,El Carmen,Manabi,C,3
90934,2017-08-14,54,El Carmen,Manabi,C,3


In [5]:


df_holiday_not_null=df[~df['holiday_info'].isnull()].reset_index(drop=True)

cols= ['date','store_nbr','holiday_info']

df_holiday_not_null=df_holiday_not_null[cols]

df_holiday_not_null['holiday_info']=df_holiday_not_null['holiday_info'].astype(str)
df_holiday_not_null['holiday_info']=df_holiday_not_null['holiday_info'].map(lambda x: x.lstrip('['))
df_holiday_not_null['holiday_info']=df_holiday_not_null['holiday_info'].map(lambda x: x.rstrip(']'))
df_holiday_not_null['holiday_info']=df_holiday_not_null['holiday_info'].map(lambda x: x.replace('false','"false"'))
df_holiday_not_null['holiday_info']=df_holiday_not_null['holiday_info'].map(lambda x: x.replace('true','"true"'))

df_holiday_not_null['holiday_info']=df_holiday_not_null['holiday_info'].map(eval)
holiday_df = pd.json_normalize(df_holiday_not_null['holiday_info'])

processed_holiday=pd.concat([df_holiday_not_null,holiday_df],axis=1)
processed_holiday=processed_holiday.drop(['holiday_info'],axis=1)


processed_holiday




,date,store_nbr,type,locale,locale_name,description,transferred
0,2013-01-01,1,Holiday,National,Ecuador,Primer dia del ano,false
1,2013-01-05,1,Work Day,National,Ecuador,Recupero puente Navidad,false
2,2013-01-12,1,Work Day,National,Ecuador,Recupero puente primer dia del ano,false
3,2013-02-11,1,Holiday,National,Ecuador,Carnaval,false
4,2013-02-12,1,Holiday,National,Ecuador,Carnaval,false
...,...,...,...,...,...,...,...
13603,2017-07-25,54,Additional,Local,Guayaquil,Fundacion de Guayaquil,false
13604,2017-08-05,54,Holiday,Local,Esmeraldas,Fundacion de Esmeraldas,false
13605,2017-08-10,54,Holiday,National,Ecuador,Primer Grito de Independencia,true
13606,2017-08-11,54,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,false


In [6]:
merge_store=pd.merge(df_base,processed_store,on=['date','store_nbr'],how='inner')

merge_holiday=pd.merge(merge_store,processed_holiday,on=['date','store_nbr'],how='left').rename(columns={'type_x':'store_type','type_y':'holiday_type'})
merge_holiday


,date,store_nbr,sales,promotion_count,city,state,store_type,CLUSTER,holiday_type,locale,locale_name,description,transferred
0,2013-01-01,1,0.000000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,false
1,2013-01-02,1,7417.148000,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,1,5873.244001,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,1,5919.879001,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,1,6318.785010,0,Quito,Pichincha,D,13,Work Day,National,Ecuador,Recupero puente Navidad,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90931,2017-08-11,54,8513.834000,166,El Carmen,Manabi,C,3,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,false
90932,2017-08-12,54,9139.678002,138,El Carmen,Manabi,C,3,NaN,NaN,NaN,NaN,NaN
90933,2017-08-13,54,14246.827996,173,El Carmen,Manabi,C,3,NaN,NaN,NaN,NaN,NaN
90934,2017-08-14,54,11882.994000,126,El Carmen,Manabi,C,3,NaN,NaN,NaN,NaN,NaN


In [11]:
interim_df=merge_holiday.copy()

interim_df['date']=pd.to_datetime(interim_df['date'], format="%Y-%m-%d")

interim_df['year']=interim_df['date'].dt.strftime('%Y').astype(int)

interim_df['month']=interim_df['date'].dt.strftime('%m').astype(int)

interim_df['day_of_mnth']=interim_df['date'].dt.strftime('%d').astype(int)

interim_df['day_of_week']=interim_df['date'].dt.strftime('%a')

interim_df['daynum_of_week']=interim_df['date'].dt.strftime('%w').astype(int)

interim_df['mnth_name']=interim_df['date'].dt.strftime('%b')



prepped_data.to_csv('store_sales_data_processed.csv',index=False)

In [10]:
prepped_data

,date,store_nbr,sales,promotion_count,city,state,store_type,CLUSTER,holiday_type,locale,locale_name,description,transferred,year,month,day_of_mnth,day_of_week,daynum_of_week,mnth_name
0,2013-01-01,1,0.000000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,false,2013,1,1,Tue,2,Jan
1,2013-01-02,1,7417.148000,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2013,1,2,Wed,3,Jan
2,2013-01-03,1,5873.244001,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2013,1,3,Thu,4,Jan
3,2013-01-04,1,5919.879001,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2013,1,4,Fri,5,Jan
4,2013-01-05,1,6318.785010,0,Quito,Pichincha,D,13,Work Day,National,Ecuador,Recupero puente Navidad,false,2013,1,5,Sat,6,Jan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90931,2017-08-11,54,8513.834000,166,El Carmen,Manabi,C,3,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,false,2017,8,11,Fri,5,Aug
90932,2017-08-12,54,9139.678002,138,El Carmen,Manabi,C,3,NaN,NaN,NaN,NaN,NaN,2017,8,12,Sat,6,Aug
90933,2017-08-13,54,14246.827996,173,El Carmen,Manabi,C,3,NaN,NaN,NaN,NaN,NaN,2017,8,13,Sun,0,Aug
90934,2017-08-14,54,11882.994000,126,El Carmen,Manabi,C,3,NaN,NaN,NaN,NaN,NaN,2017,8,14,Mon,1,Aug
